In [46]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# MOdel selection
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import IncrementalPCA
from sklearn.linear_model import LogisticRegression

# Model hyper parameter tuning
from sklearn import metrics
from mpl_toolkits.mplot3d import Axes3D
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_score, recall_score, f1_score, roc_auc_score,roc_curve

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import os


import warnings  
warnings.filterwarnings('ignore')
from sklearn.neighbors import LocalOutlierFactor

In [3]:
def read_file(folder, samp = False, lof=False):
    df = pd.DataFrame()
    path = folder+'/'
    df_new = pd.read_csv(path+'benign_traffic.csv')
    df_new['label'] = 'benign_traffic'
    if samp==True:
        df_new = df_new.sample(frac=0.5, replace=False, random_state=42).reset_index(drop=True)
    if lof==True:
        #df_new = pd.DataFrame(scaler.fit_transform(df_new))
        df_new = filter_lof(df_new)
        #df_new = scaler.inverse_transform(df_new)
    df = pd.concat([df,df_new], ignore_index=True)
    
    directory = os.fsencode(path+'gafgyt_attacks')
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.endswith(".csv"): 
            df_new = pd.read_csv(path+'gafgyt_attacks'+'/'+filename)
            filename=filename[:-4] 
            df_new['label'] = 'gafgyt_'+filename
            if samp==True:
                df_new = df_new.sample(frac=0.5, replace=False, random_state=42).reset_index(drop=True)
            if lof==True:
                #df_new = pd.DataFrame(scaler.fit_transform(df_new))
                df_new = filter_lof(df_new)
                #df_new = scaler.inverse_transform(df_new)
            df = pd.concat([df,df_new], ignore_index=True)
    directory = os.fsencode(path+'mirai_attacks')
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.endswith(".csv"): 
            df_new = pd.read_csv(path+'mirai_attacks'+'/'+filename)
            filename=filename[:-4] 
            df_new['label'] = 'mirai_'+filename
            if samp==True:
                df_new = df_new.sample(frac=0.5, replace=False, random_state=42).reset_index(drop=True)
            if lof==True:
                #df_new = pd.DataFrame(scaler.fit_transform(df_new))
                df_new = filter_lof(df_new)
                #df_new = scaler.inverse_transform(df_new)
            df = pd.concat([df,df_new], ignore_index=True)
    return df

In [43]:
def filter_lof(df, k=20) :
    lof = LocalOutlierFactor(n_neighbors=k)
    scaler = StandardScaler()
    df2 = pd.DataFrame.copy(df)
    df2 = df2.drop(columns = ['label'])
    df2 = pd.DataFrame(scaler.fit_transform(df2))
    df2["_lof"] = lof.fit_predict(df2)
    df2 = df2[df2["_lof"]>0].drop(columns="_lof")#.reset_index(drop=True)
    df2 = pd.DataFrame(scaler.inverse_transform(df2))
    df2['label'] = df['label']
    return df2.reset_index(drop=True)

In [40]:
def churn_prediction(algorithm,training_x,testing_x,
                             training_y,testing_y,cols,cf,threshold_plot) :
    
    #model
    algorithm.fit(training_x,training_y)
    predictions   = algorithm.predict(testing_x)
    probabilities = algorithm.predict_proba(testing_x)
    #coeffs
    if   cf == "coefficients" :
        coefficients  = pd.DataFrame(algorithm.coef_.ravel())
    elif cf == "features" :
        coefficients  = pd.DataFrame(algorithm.feature_importances_)
        
    column_df     = pd.DataFrame(cols)
    coef_sumry    = (pd.merge(coefficients,column_df,left_index= True,
                              right_index= True, how = "left"))
    coef_sumry.columns = ["coefficients","features"]
    coef_sumry    = coef_sumry.sort_values(by = "coefficients",ascending = False)
    
    #print (algorithm)
    #print ("\n Classification report : \n",classification_report(testing_y,predictions))
    #print ("Accuracy   Score : ",accuracy_score(testing_y,predictions))
    return accuracy_score(testing_y,predictions)
    #confusion matrix
    conf_matrix = confusion_matrix(testing_y,predictions)
    '''
    #roc_auc_score
    model_roc_auc = roc_auc_score(testing_y,predictions) 
    print ("Area under curve : ",model_roc_auc,"\n")
    fpr,tpr,thresholds = roc_curve(testing_y,probabilities[:,1])
    #plot confusion matrix
    trace1 = go.Heatmap(z = conf_matrix ,
                        x = ["Not churn","Churn"],
                        y = ["Not churn","Churn"],
                        showscale  = False,colorscale = "Picnic",
                        name = "matrix")
    
    #plot roc curve
    trace2 = go.Scatter(x = fpr,y = tpr,
                        name = "Roc : " + str(model_roc_auc),
                        line = dict(color = ('rgb(22, 96, 167)'),width = 2))
    trace3 = go.Scatter(x = [0,1],y=[0,1],
                        line = dict(color = ('rgb(205, 12, 24)'),width = 2,
                        dash = 'dot'))
    
    #plot coeffs
    trace4 = go.Bar(x = coef_sumry["features"],y = coef_sumry["coefficients"],
                    name = "coefficients",
                    marker = dict(color = coef_sumry["coefficients"],
                                  colorscale = "Picnic",
                                  line = dict(width = .6,color = "black")))
    
    #subplots
    fig = tls.make_subplots(rows=2, cols=2, specs=[[{}, {}], [{'colspan': 2}, None]],
                            subplot_titles=('Confusion Matrix',
                                            'Receiver operating characteristic',
                                            'Feature Importances'))
    
    fig.append_trace(trace1,1,1)
    fig.append_trace(trace2,1,2)
    fig.append_trace(trace3,1,2)
    fig.append_trace(trace4,2,1)
    
    fig['layout'].update(showlegend=False, title="Model performance" ,
                         autosize = False,height = 900,width = 800,
                         plot_bgcolor = 'rgba(240,240,240, 0.95)',
                         paper_bgcolor = 'rgba(240,240,240, 0.95)',
                         margin = dict(b = 195))
    fig["layout"]["xaxis2"].update(dict(title = "false positive rate"))
    fig["layout"]["yaxis2"].update(dict(title = "true positive rate"))
    fig["layout"]["xaxis3"].update(dict(showgrid = True,tickfont = dict(size = 10),
                                        tickangle = 90))
    py.iplot(fig)
    
    if threshold_plot == True : 
        visualizer = DiscriminationThreshold(algorithm)
        visualizer.fit(training_x,training_y)
        visualizer.poof()
    '''

In [54]:
def train_model(filename, algo, seed) :
    X, Y = readXY(filename)
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=seed)
    rfe_algo = algo
    
    accuracy = churn_prediction(rfe_algo, x_train, x_test, y_train, y_test, x_train.columns,"features",threshold_plot = False)
    return rfe_algo, accuracy

In [70]:
def readXY(filename) :
    df = read_file(filename, samp=True, lof=False)
    df = df.dropna().reset_index(drop=True)
    X = df.drop(columns=['label'], axis=1)
    Y = df[['label']]
    return X, Y

In [36]:
def churn_validation(algorithm,testing_x,
                             testing_y,cols) :
    #model
    predictions   = algorithm.predict(testing_x)
    probabilities = algorithm.predict_proba(testing_x)
    
    #print (algorithm)
    #print ("\n Classification report : \n",classification_report(testing_y,predictions))
    #print ("Accuracy   Score : ",accuracy_score(testing_y,predictions))
    return accuracy_score(testing_y,predictions)
    #confusion matrix
    #conf_matrix = confusion_matrix(testing_y,predictions)
    '''
    #roc_auc_score
    model_roc_auc = roc_auc_score(testing_y,predictions) 
    print ("Area under curve : ",model_roc_auc,"\n")
    fpr,tpr,thresholds = roc_curve(testing_y,probabilities[:,1])
    
    #plot confusion matrix
    trace1 = go.Heatmap(z = conf_matrix ,
                        x = ["Not churn","Churn"],
                        y = ["Not churn","Churn"],
                        showscale  = False,colorscale = "Picnic",
                        name = "matrix")
    
    #plot roc curve
    trace2 = go.Scatter(x = fpr,y = tpr,
                        name = "Roc : " + str(model_roc_auc),
                        line = dict(color = ('rgb(22, 96, 167)'),width = 2))
    trace3 = go.Scatter(x = [0,1],y=[0,1],
                        line = dict(color = ('rgb(205, 12, 24)'),width = 2,
                        dash = 'dot'))
    #subplots
    fig = tls.make_subplots(rows=2, cols=2, specs=[[{}, {}], [{'colspan': 2}, None]],
                            subplot_titles=('Confusion Matrix',
                                            'Receiver operating characteristic'))
    
    fig.append_trace(trace1,1,1)
    fig.append_trace(trace2,1,2)
    fig.append_trace(trace3,1,2)
    
    fig['layout'].update(showlegend=False, title="Model performance" ,
                         autosize = False,height = 900,width = 800,
                         plot_bgcolor = 'rgba(240,240,240, 0.95)',
                         paper_bgcolor = 'rgba(240,240,240, 0.95)',
                         margin = dict(b = 195))
    fig["layout"]["xaxis2"].update(dict(title = "false positive rate"))
    fig["layout"]["yaxis2"].update(dict(title = "true positive rate"))
    fig["layout"]["xaxis3"].update(dict(showgrid = True,tickfont = dict(size = 10),
                                        tickangle = 90))
    py.iplot(fig)
    '''

In [64]:
def validate_with_raw_data(model, device) :
    X, Y = readXY('/data/dataprivacy/detection_of_IoT_botnet_attacks/'+device)
    acc = churn_validation(model, X,Y, X.columns)
    return acc

# Pre-processed, NON-anonymized data (reference)

In [57]:
device = 'Philips_B120N10_Baby_Monitor'
filename = '/data/dataprivacy/detection_of_IoT_botnet_attacks/'+device
algorithm = RandomForestClassifier(n_jobs=-1,
                                bootstrap=True,
                                max_depth=10,
                                min_samples_leaf=50,
                                min_samples_split=50,
                                n_estimators=60)
model, acc =  train_model(filename, algorithm, 0)

# Training with pre-processed & anonymized data

In [41]:
device = 'Philips_B120N10_Baby_Monitor'
algo = 'alex2'
file = 'baby_k10_e1_lof'
filename = '/data/dataprivacy/detection_of_IoT_botnet_attacks-anonymized/'+device+'/'+algo+'/'+file
algorithm = RandomForestClassifier(n_jobs=-1,
                                bootstrap=True,
                                max_depth=10,
                                min_samples_leaf=50,
                                min_samples_split=50,
                                n_estimators=60)
model =  train_model(filename, algorithm)

# Validate with original data

In [65]:
validate_with_raw_data(model, device)

0.9917938760830626

In [48]:
algos = [RandomForestClassifier(n_jobs=-1,
                                bootstrap=True,
                                max_depth=10,
                                min_samples_leaf=50,
                                min_samples_split=50,
                                n_estimators=60)]

In [73]:
df_eval = pd.DataFrame(columns = ['accuracy'])
#for device in ['SimpleHome_XCS7_1003_WHT_Security_Camera', 'Philips_B120N10_Baby_Monitor']:
for device in ['Philips_B120N10_Baby_Monitor']:
    print(device)
    #original file beolvasas
    files = ['benign_traffic', 'bashlite_combo', 'bashlite_junk', 'bashlite_scan', 'bashlite_tcp', 'bashlite_udp', 'mirai_ack', 'mirai_scan', 'mirai_syn', 'mirai_udp', 'mirai_udpplain']
    #for algo_anon in ['alex2', 'attila1', 'attila2']:
    for algo_anon in ['alex1b','alex2']:
        print(algo_anon)
        if algo_anon=='alex1b' or 'alex2':
            df_original = read_file('/data/dataprivacy/detection_of_IoT_botnet_attacks/'+device, samp=True, lof=True)
            df_original = df_original.dropna().reset_index(drop=True)
        if algo_anon=='attila1' or 'attila2':
            df_original = read_file('/data/dataprivacy/detection_of_IoT_botnet_attacks/'+device, samp=True)
        for file in os.listdir('/data/dataprivacy/detection_of_IoT_botnet_attacks-anonymized/'+device+'/'+algo_anon):
            if file.startswith('c') or file.startswith('k') or (file.startswith('b') and file.endswith('lof')):
                print(file)
                #file beolvasas
                folder = '/data/dataprivacy/detection_of_IoT_botnet_attacks-anonymized/'+device+'/'+algo_anon+'/'+file
                #df = read_file(folder)
                #df = df.dropna().reset_index(drop=True)
                ultim_best_scores = 0
                df_ultim_best = pd.DataFrame()
                for seed in [0,1,2,3]:
                    print(seed)
                    ultim_best_scores = 0
                    df_ultim_best = pd.DataFrame()
                    for algo in algos:
                        print(algo)
                        model, accuracy_train =  train_model(folder, algo, seed)
                        if accuracy_train > ultim_best_scores:
                            ultim_best_scores = accuracy_train
                            best_model = model 
    
                    ###
                    df_eval.loc[device+'_'+algo_anon+'_'+file+'_'+str(seed)] = ultim_best_scores
                    df_eval.loc[device+'_'+algo_anon+'_'+file+'_'+str(seed)+'_original'] = validate_with_raw_data(best_model, device)
                    df_eval.to_csv('eval_total_class_alex_'+device+'.csv')

Philips_B120N10_Baby_Monitor
alex1b
k10_e10_lof
0
RandomForestClassifier(max_depth=10, min_samples_leaf=50, min_samples_split=50,
                       n_estimators=60, n_jobs=-1)
1
RandomForestClassifier(max_depth=10, min_samples_leaf=50, min_samples_split=50,
                       n_estimators=60, n_jobs=-1)
2
RandomForestClassifier(max_depth=10, min_samples_leaf=50, min_samples_split=50,
                       n_estimators=60, n_jobs=-1)
3
RandomForestClassifier(max_depth=10, min_samples_leaf=50, min_samples_split=50,
                       n_estimators=60, n_jobs=-1)
k10_e1_lof
0
RandomForestClassifier(max_depth=10, min_samples_leaf=50, min_samples_split=50,
                       n_estimators=60, n_jobs=-1)
1
RandomForestClassifier(max_depth=10, min_samples_leaf=50, min_samples_split=50,
                       n_estimators=60, n_jobs=-1)
2
RandomForestClassifier(max_depth=10, min_samples_leaf=50, min_samples_split=50,
                       n_estimators=60, n_jobs=-1)
3
RandomFo

baby_k50_e10_lof
0
RandomForestClassifier(max_depth=10, min_samples_leaf=50, min_samples_split=50,
                       n_estimators=60, n_jobs=-1)
1
RandomForestClassifier(max_depth=10, min_samples_leaf=50, min_samples_split=50,
                       n_estimators=60, n_jobs=-1)
2
RandomForestClassifier(max_depth=10, min_samples_leaf=50, min_samples_split=50,
                       n_estimators=60, n_jobs=-1)
3
RandomForestClassifier(max_depth=10, min_samples_leaf=50, min_samples_split=50,
                       n_estimators=60, n_jobs=-1)
baby_k20_e10_lof
0
RandomForestClassifier(max_depth=10, min_samples_leaf=50, min_samples_split=50,
                       n_estimators=60, n_jobs=-1)
1
RandomForestClassifier(max_depth=10, min_samples_leaf=50, min_samples_split=50,
                       n_estimators=60, n_jobs=-1)
2
RandomForestClassifier(max_depth=10, min_samples_leaf=50, min_samples_split=50,
                       n_estimators=60, n_jobs=-1)
3
RandomForestClassifier(max_depth=

In [ ]:
eval_privacy(devices, algos):
    df_avg_ident = pd.DataFrame(columns = ['avg_per_10'])
    for device in devices:
        print(device)
        for algo in algos:
            print(algo)
            if algo=='attila1' or algo=='attila2':
                df_original = read_file('/data/dataprivacy/detection_of_IoT_botnet_attacks/'+device, samp=True, lof=False)
                print('attila')
            else:
                df_original = read_file('/data/dataprivacy/detection_of_IoT_botnet_attacks/'+device, samp=True, lof=True)
                print('nemattila')
            for file in os.listdir('/data/dataprivacy/detection_of_IoT_botnet_attacks-anonymized/'+device+'/'+algo):
                #if file.startswith('c') or file.startswith('k'):
                if file.endswith('lof'):
                    print(file)
                    if algo=='attila1' or algo=='attila2':
                        if int(file[1])==3 or int(file[1])==5:
                            num = int(file[1:2])
                        else:
                            num = int(file[1:3])
                        print(num)
                        df_original_cropped = crop_attila(df_original, num, device)
                        df = read_file('/data/dataprivacy/detection_of_IoT_botnet_attacks-anonymized/'+device+'/'+algo+'/'+file)
                        print(len(df_original_cropped))
                        print(len(df))
                        for j in df_original_cropped['label'].unique():
                            if df_original_cropped[df_original_cropped['label'] == j].shape[0] != df[df['label'] == j].shape[0]:
                                print('baaaj')
                                print(j)
                                print(df_original_cropped[df_original_cropped['label'] == j].shape[0])
                                print(df[df['label'] == j].shape[0])
                        X = df_original_cropped.iloc[:,:-1]
                    else:
                        df = read_file('/data/dataprivacy/detection_of_IoT_botnet_attacks-anonymized/'+device+'/'+algo+'/'+file)
                        print(len(df_original))
                        print(len(df))
                        for j in df_original['label'].unique():
                            if df_original[df_original['label'] == j].shape[0] != df[df['label'] == j].shape[0]:
                                print('baaaaaaaaaaaaaaaaaaaaaaaaaaj')
                                print(j)
                                print(df_original[df_original['label'] == j].shape[0])
                                print(df[df['label'] == j].shape[0])
                        X = df_original.iloc[:,:-1]
                    nbrs_original = NearestNeighbors(n_neighbors=11, algorithm='ball_tree').fit(X)
                    distances_original, indices_original = nbrs_original.kneighbors(X)                    
                    indices_original = indices_original[:,1:]
                    inds = pd.isnull(df).any(1).to_numpy().nonzero()
                    df = df.dropna().reset_index(drop=True)
                    X = X.drop(inds[0]).reset_index(drop=True)
                    Y = df.iloc[:,:-1]
                    ##inds = pd.isnull(Y).any(1).nonzero()[0]
                    #Y = Y.dropna().reset_index(drop=True)
                    #X = X.drop(inds).reset_index(drop=True)
                    print(len(X))
                    print(len(Y))
                    nbrs = NearestNeighbors(n_neighbors=11, algorithm='ball_tree').fit(Y)
                    distances, indices = nbrs.kneighbors(Y)
                    indices = indices[:,1:]
                    identical = []
                    for l in range(len(indices)):
                        identical.append(len(set(indices_original[l]).intersection(indices[l])))
                    df_avg_ident.loc[device+'_'+algo+'_'+file] = sum(identical)/len(identical)
                    df_avg_ident.to_csv('eval_privacy_baby_alex2.csv')